## Imports

In [35]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Data read-in

In [36]:
df = pd.read_csv('Tides Texas Properties - Sheet1.csv', converters={'Year built':str})

## Map prep

In [37]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [38]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Address'].apply(geocode)

In [39]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [40]:
df['Year built'] = df['Year built'].astype(str)

In [41]:
def popup_html(row):
    entity_name = row['Ownership entity']
    address = row['Address']
    units = row['# Units']
    sqft = row['Square feet']
    mrkt_value = row['Market value']
    year_built = row['Year built']

    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Entity: </strong>{}'''.format(entity_name) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong># Units: </strong>{}'''.format(units) + '''<br>
    <strong>Square Footage: </strong>{}'''.format(sqft) + '''<br>
    <strong>Market Value: </strong>{}'''.format(mrkt_value) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    </html>
    '''
    return html

In [42]:
import folium
from folium.plugins import MarkerCluster

# m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=16)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Tides Texas Properties')

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=6,tiles='OpenStreetMap')

for index, row in df.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m

In [43]:
m.save('index.html')

## Map URL Snagger

In [44]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/TidesTexasProperties
